So how does one use this script?

Note that this is a bastardized version of https://colab.research.google.com/drive/117seAcMrQvapom62Ux9CN9TPrxWoRy22?usp=sharing so if You are confused by something, it might be worth it to find the same line in the old code for a hint.

The script _seems_ huge but most if it can be readily ignored. It's a lot easier to get this to work than it might seem.

1. You'll need to get /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz to exist in Your gdrive directory.

  You can run bash commands using ! in google colab (so !ls and !cd etc), so just keep messing with stuff until You are able to run !tar -xzvf /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/

  This will really just involve creating a darknet folder in Your Gdrive and putting cudnn-10.1-linux-x64-v7.6.5.32.tgz in there. Google the filename to find a download link.



2. You'll need access to a bunch of google directories. An easy way of going about it is just stepping through the script until You get a "missing" error. 

  However, You can immediately request access to the following
  https://drive.google.com/drive/u/0/folders/17mTQanLeTbYrBmIInKmp3PVgqdMuRxV_
  https://drive.google.com/drive/u/0/folders/1Vx5gmORtJ2Q7Eprs3Ei4aMwwfOsKp4M9
  https://drive.google.com/drive/u/0/folders/1GEhPtvNA74kjnjpNH_3FtrySVXIIiXnm
  https://drive.google.com/drive/u/0/folders/1ur2tAvROwtB0i02gEGKra35nD_jiasX9

3. Configuring paths

  Everything You'll access will be relative to Your google drive, so You'll have to reconfigure the paths to reflect that, i.e. paths will now have to use YOUR shortcuts to shared folders etc. It's annoying, I know.

  So when I had to do this, accessing shared stuff via terminal was a pain and I give or take followed https://stackoverflow.com/a/62055342, that's why You see a bunch of '/content/gdrive/.shortcut-targets-by-id' paths. Creating shortcuts and just using their names might work now, but if it does not then You'll have to hardcode in Your shortcut links instead. This will just involve right clicking the shared folder in Google Drive and clicking "create shortcut". When You run !ls -a /content/gdrive/ it should show Your .shortcut-targets-by-id folder, which contains the shortcuts.

Once all the paths have been configured, it should just run. If You ctrl+f for "scaling" You'll find that We enlarged the YOLO output boxes by a factor of 2. Change the scaling variable for a different result, of course.

Otherwise it does mostly what it says on the tin, e.g. 

4. Using this cursed thing.

  trained_models contains a list of models You want to use to generate bounding boxes - for us this was some folder of pretrained stuff. Names in trained_models are appended on to a path to read in the weights files later.

  Anywho, the weights are read in and using an API for darknet the model is used to generate bounding boxes for images. The images in this case are frames from the video. ctrl+f "every n'th frame" and change the modulus number to get, well, different every n'th frame. 

  If any of this is confusing it's because it is - spend maybe 20 minutes on a particular weird bit of code and if it's still not working contact me at andriusb@ethz.ch to save Yourself some pain.



  


Note: if You run into stuff like "Mish activation missing" etc, it's because Your openCV version is too old. Unfortunately, every time a google colab notebook spins up, it's with an old version of openCV, so, You have to recompile the damn thing every time (at least as of the writing of this in January 2021). Sometimes the machine shuts down and You restart it without noticing, resulting in an old OpenCV version. 



In [ ]:
!ls -a /content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/
!ls -a /content/gdrive/

beeTracking_img_labeling
.file-revisions-by-id  MyDrive	Shareddrives  .shortcut-targets-by-id  .Trash


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [ ]:
!ls /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz

'/content/gdrive/My Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz'


In [ ]:
# Unzipping the cuDNN files from your Drive folder directly to the VM CUDA folders
#!tar -xzvf /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/ #gabriela
#!tar -xzvf /content/gdrive/My\ Drive/Bees/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/ #pascal
!tar -xzvf /content/gdrive/My\ Drive/darknet/cuDNN/cudnn-10.1-linux-x64-v7.6.5.32.tgz -C /usr/local/

!chmod a+r /usr/local/cuda/include/cudnn.h

# Now we check the version we already installed. Can comment this line on future runs
!cat /usr/local/cuda/include/cudnn.h | grep CUDNN_MAJOR -A 2

cuda/include/cudnn.h
cuda/NVIDIA_SLA_cuDNN_Support.txt
cuda/lib64/libcudnn.so
cuda/lib64/libcudnn.so.7
cuda/lib64/libcudnn.so.7.6.5
cuda/lib64/libcudnn_static.a
#define CUDNN_MAJOR 7
#define CUDNN_MINOR 6
#define CUDNN_PATCHLEVEL 5
--
#define CUDNN_VERSION (CUDNN_MAJOR * 1000 + CUDNN_MINOR * 100 + CUDNN_PATCHLEVEL)

#include "driver_types.h"


# Import CV2 Version 4.X to use Mish Activation

In [ ]:
from time import gmtime, strftime
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-11-01 09:58:11


In [ ]:
# compile cv 4.3 in colab
!git clone https://github.com/opencv/opencv.git
!git clone https://github.com/opencv/opencv_contrib.git
!pip uninstall opencv-python
!pip uninstall opencv-contrib-python

Cloning into 'opencv'...
remote: Enumerating objects: 5, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 281399 (delta 0), reused 1 (delta 0), pack-reused 281394
Receiving objects: 100% (281399/281399), 473.47 MiB | 33.33 MiB/s, done.
Resolving deltas: 100% (196586/196586), done.
Checking out files: 100% (6654/6654), done.
Cloning into 'opencv_contrib'...
remote: Enumerating objects: 15, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 33272 (delta 3), reused 8 (delta 3), pack-reused 33257
Receiving objects: 100% (33272/33272), 129.59 MiB | 37.46 MiB/s, done.
Resolving deltas: 100% (20534/20534), done.
Uninstalling opencv-python-4.1.2.30:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/cv2/*
    /usr/local/lib/python3.6/dist-packages/opencv_python-4.1.2.30.dist-info/*
Proceed (y/n)? y
  Successfully uninstalled opencv-python-4.1.2.30
Uninstalling o

In [ ]:
%cd opencv
!mkdir build
%cd build

/content/opencv
/content/opencv/build


In [ ]:
!git describe --tags

4.5.0-76-g691c3d1e3c


In [ ]:
!cmake -DCMAKE_BUILD_TYPE=Release -DBUILD_opencv_cudacodec=OFF -DOPENCV_EXTRA_MODULES_PATH=/content/opencv_contrib/modules  -D BUILD_DOCS=OFF -D BUILD_EXAMPLES=OFF -D BUILD_TIFF=OFF -D BUILD_JPEG=ON -D BUILD_PNG=ON -D BUILD_PERF_TESTS=OFF -D BUILD_TESTS=OFF -D BUILD_JAVA=OFF -D BUILD_opencv_js=OFF -D BUILD_opencv_ts=OFF -D WITH_CUDA=ON -D WITH_CUFFT=ON -D ENABLE_FAST_MATH=1 -D CUDA_FAST_MATH=1 -D INSTALL_PYTHON_EXAMPLES=OFF -DCUDA_ARCH_BIN=75 ..

-- The CXX compiler identification is GNU 7.5.0
-- The C compiler identification is GNU 7.5.0
-- Check for working CXX compiler: /usr/bin/c++
-- Check for working CXX compiler: /usr/bin/c++ -- works
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Detecting CXX compile features
-- Detecting CXX compile features - done
-- Check for working C compiler: /usr/bin/cc
-- Check for working C compiler: /usr/bin/cc -- works
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Detecting C compile features
-- Detecting C compile features - done
-- Detected processor: x86_64
-- Found PythonInterp: /usr/bin/python2.7 (found suitable version "2.7.17", minimum required is "2.7") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython2.7.so (found suitable exact version "2.7.17") 
-- Found PythonInterp: /usr/bin/python3 (found suitable version "3.6.9", minimum required is "3.2") 
-- Found PythonLibs: /usr/lib/x86_64-linux-gnu/libpython3.6m.so (fo

In [ ]:
!nproc

2


In [ ]:
!make -j2
!make install

Scanning dependencies of target libjpeg-turbo
Scanning dependencies of target libwebp
[  0%] Building C object 3rdparty/libjpeg-turbo/CMakeFiles/libjpeg-turbo.dir/src/jcapimin.c.o
[  0%] Building C object 3rdparty/libwebp/CMakeFiles/libwebp.dir/src/dec/alpha_dec.c.o
[  0%] Building C object 3rdparty/libjpeg-turbo/CMakeFiles/libjpeg-turbo.dir/src/jcapistd.c.o
[  0%] Building C object 3rdparty/libwebp/CMakeFiles/libwebp.dir/src/dec/buffer_dec.c.o
[  0%] Building C object 3rdparty/libjpeg-turbo/CMakeFiles/libjpeg-turbo.dir/src/jccoefct.c.o
[  0%] Building C object 3rdparty/libwebp/CMakeFiles/libwebp.dir/src/dec/frame_dec.c.o
[  0%] Building C object 3rdparty/libjpeg-turbo/CMakeFiles/libjpeg-turbo.dir/src/jccolor.c.o
[  0%] Building C object 3rdparty/libwebp/CMakeFiles/libwebp.dir/src/dec/idec_dec.c.o
[  0%] Building C object 3rdparty/libjpeg-turbo/CMakeFiles/libjpeg-turbo.dir/src/jcdctmgr.c.o
[  0%] Building C object 3rdparty/libwebp/CMakeFiles/libwebp.dir/src/dec/io_dec.c.o
[  0%] Buildi

In [ ]:
!ldconfig -v

/sbin/ldconfig.real: Path `/usr/local/lib' given more than once
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/nvidia/lib64: No such file or directory
/sbin/ldconfig.real: Can't stat /usr/local/lib/x86_64-linux-gnu: No such file or directory
/sbin/ldconfig.real: Path `/lib/x86_64-linux-gnu' given more than once
/sbin/ldconfig.real: Path `/usr/lib/x86_64-linux-gnu' given more than once
/usr/local/cuda-10.1/extras/CUPTI/lib64:
	libnvperf_target.so -> libnvperf_target.so
	libnvperf_host.so -> libnvperf_host.so
	libcupti.so.10.1 -> libcupti.so.10.1.208
/usr/local/lib:
	libmkl_mc3.so -> libmkl_mc3.so
	libmkl_intel_lp64.so -> libmkl_intel_lp64.so
	libmkl_vml_mc2.so -> libmkl_vml_mc2.so
	libmkl_scalapack_lp64.so -> libmkl_scalapack_lp64.so
	libmkl_avx512.so -> libmkl_avx512.so
	libmkl_cdft_core.so -> libmkl_cdft_core.so
	libmkl_sequential.so -> libmkl_sequential.so
	libmkl_def.so -> libmkl_def.so
	libmkl_tbb_thread.s

In [ ]:

import cv2
print(cv2.__version__)

4.1.2


In [ ]:
print(strftime("%Y-%m-%d %H:%M:%S", gmtime()))

2020-11-01 11:06:54


In [ ]:
print(cv2.getBuildInformation())


General configuration for OpenCV 4.5.0-dev =====================================
  Version control:               4.5.0-76-g691c3d1e3c

  Extra modules:
    Location (extra):            /content/opencv_contrib/modules
    Version control (extra):     4.5.0-13-gb65b3cc3

  Platform:
    Timestamp:                   2020-11-01T10:03:04Z
    Host:                        Linux 4.19.112+ x86_64
    CMake:                       3.12.0
    CMake generator:             Unix Makefiles
    CMake build tool:            /usr/bin/make
    Configuration:               Release

  CPU/HW features:
    Baseline:                    SSE SSE2 SSE3
      requested:                 SSE3
    Dispatched code generation:  SSE4_1 SSE4_2 FP16 AVX AVX2 AVX512_SKX
      requested:                 SSE4_1 SSE4_2 AVX FP16 AVX2 AVX512_SKX
      SSE4_1 (15 files):         + SSSE3 SSE4_1
      SSE4_2 (1 files):          + SSSE3 SSE4_1 POPCNT SSE4_2
      FP16 (0 files):            + SSSE3 SSE4_1 POPCNT SSE4_2 FP16 AVX


In [ ]:
import re

cv_info = [re.sub('\s+', ' ', ci.strip()) for ci in cv2.getBuildInformation().strip().split('\n') 
               if len(ci) > 0 and re.search(r'(nvidia*:?)|(cuda*:)|(cudnn*:)', ci.lower()) is not None]
print(cv_info)

['NVIDIA CUDA: YES (ver 10.1, CUFFT CUBLAS FAST_MATH)', 'NVIDIA GPU arch: 75', 'NVIDIA PTX archs:', 'cuDNN: YES (ver 7.6.5)']


# Functions

In [ ]:
#!pip install --force https://github.com/chengs/tqdm/archive/colab.zip
import numpy as np
import os
import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm import tqdm
import sqlite3
from shapely.geometry import Polygon
from copy import deepcopy
import pandas as pd
import seaborn as sns
import imutils
import time

import os.path
from numpy import genfromtxt

import pandas as pd

import xml.etree.ElementTree as ET

In [ ]:
import xml.etree.ElementTree as xmlbuilder
import xml.dom.minidom


def create_xml_voc_files(bboxes, gt_bboxes, img_sizes, xml_folder_path):
  root = xmlbuilder.Element("annotation")
  fldr = xmlbuilder.Element("folder")
  fldr.text = "validate"
  root.append(fldr)

  for img_file in bboxes.keys():
    xml_filename = xml_folder_path + img_file.split('.')[0] + '.xml'

    #exit if file exists
    if os.path.isfile(xml_filename):
      return

    flnm = xmlbuilder.Element("filename")
    flnm.text = img_file
    root.append(flnm)
    pth = xmlbuilder.Element("path")
    pth.text = img_file
    root.append(pth)

    src = xmlbuilder.Element("source")
    db = xmlbuilder.SubElement(src, "database")
    db.text = "Unknown"
    root.append(src)

    sz = xmlbuilder.Element("size")
    wd = xmlbuilder.SubElement(sz, "width")
    wd.text = str(img_sizes[0][0])
    hg = xmlbuilder.SubElement(sz, "height")
    hg.text = str(img_sizes[0][1])
    dp = xmlbuilder.SubElement(sz, "depth")
    dp.text = str(3)
    root.append(sz)

    for box in bboxes[img_file]['boxes']:
      detected_bee = xmlbuilder.Element("object")
      db_name = xmlbuilder.SubElement(detected_bee, "name")
      db_name.text = "detected_bee"
      db_pose = xmlbuilder.SubElement(detected_bee, "pose")
      db_pose.text = "Unspecified"
      db_truncated = xmlbuilder.SubElement(detected_bee, "truncated")
      db_truncated.text = str(0)
      db_difficult = xmlbuilder.SubElement(detected_bee, "difficult")
      db_difficult.text = str(0)
      db_bndbox = xmlbuilder.SubElement(detected_bee, "bndbox")
      db_bndbox_xmin = xmlbuilder.SubElement(db_bndbox, "xmin")
      db_bndbox_xmin.text = str(box[0])
      db_bndbox_ymin = xmlbuilder.SubElement(db_bndbox, "ymin")
      db_bndbox_ymin.text = str(box[1])
      db_bndbox_xmax = xmlbuilder.SubElement(db_bndbox, "xmax")
      db_bndbox_xmax.text = str(box[2])
      db_bndbox_ymax = xmlbuilder.SubElement(db_bndbox, "ymax")
      db_bndbox_ymax.text = str(box[3])
      root.append(detected_bee)

    for box in gt_bboxes[img_file]:
      gt_bee = xmlbuilder.Element("object")
      db_name = xmlbuilder.SubElement(gt_bee, "name")
      db_name.text = "gt_bee"
      db_pose = xmlbuilder.SubElement(gt_bee, "pose")
      db_pose.text = "Unspecified"
      db_truncated = xmlbuilder.SubElement(gt_bee, "truncated")
      db_truncated.text = str(0)
      db_difficult = xmlbuilder.SubElement(gt_bee, "difficult")
      db_difficult.text = str(0)
      db_bndbox = xmlbuilder.SubElement(gt_bee, "bndbox")
      db_bndbox_xmin = xmlbuilder.SubElement(db_bndbox, "xmin")
      db_bndbox_xmin.text = str(box[0])
      db_bndbox_ymin = xmlbuilder.SubElement(db_bndbox, "ymin")
      db_bndbox_ymin.text = str(box[1])
      db_bndbox_xmax = xmlbuilder.SubElement(db_bndbox, "xmax")
      db_bndbox_xmax.text = str(box[2])
      db_bndbox_ymax = xmlbuilder.SubElement(db_bndbox, "ymax")
      db_bndbox_ymax.text = str(box[3])
      root.append(gt_bee)

    tree = xmlbuilder.ElementTree(root) 
    
    xmlstr = xml.etree.ElementTree.tostring(root, encoding='utf8', method='xml')
    dom = xml.dom.minidom.parseString(xmlstr) #xml_string
    pretty_xml_as_string = dom.toprettyxml()

    with open (xml_filename, "w") as files : 
        files.write("%s\n" % pretty_xml_as_string)


In [ ]:
def CNN_get_boxes_for_frame(file,image):

  predicted = {}
  predicted[file] = {}
  predicted[file]['boxes'] = []
  predicted[file]['scores'] = []
  bboxes = []
  (W, H) = (None, None)
  no_predictions = False

  # image = frame

  # construct a blob from the input frame and then perform a forward
  # pass of the YOLO object detector, giving us our bounding boxes
  # and associated probabilities
  blob = cv2.dnn.blobFromImage(image, 1 / 255.0, (608, 608), swapRB=True, crop=False)
  net.setInput(blob)
  start = time.time()
  layerOutputs = net.forward(ln)

  if W is None or H is None:
    (H, W) = image.shape[:2]

  # loop over each of the layer outputs
  for output in layerOutputs:
    # loop over each of the detections
    for detection in output:
      # extract the class ID and confidence (i.e., probability)
      # of the current object detection
      scores = detection[5:]
      classID = np.argmax(scores)
      confidence = scores[classID]
      # filter out weak predictions by ensuring the detected
      # probability is greater than the minimum probability
      if confidence > 0.5:
        # scale the bounding box coordinates back relative to
        # the size of the image, keeping in mind that YOLO
        # actually returns the center (x, y)-coordinates of
        # the bounding box followed by the boxes' width and
        # height
        box = detection[0:4] * np.array([W, H, W, H])
        (centerX, centerY, width, height) = box.astype("int")
        # use the center (x, y)-coordinates to derive the top
        # and and left corner of the bounding box
        # scale image for pollen detection
        scaling = 2
        width = width * scaling
        height = height * scaling
        x_min = int(centerX - (width / 2))
        x_max = x_min+width
        y_min = int(centerY - (height / 2))
        y_max = y_min+height
        # update our list of bounding box coordinates,
        # confidences, and class IDs
        #boxes.append([x, y, int(width), int(height)])
        #confidences.append(float(confidence))
        #classIDs.append(classID)
        #boxes_to_frames.append([framesCaptured,x, y, int(width), int(height),confidence])
        predicted[file]['boxes'].append([x_min, y_min, x_max, y_max])
        predicted[file]['scores'].append(float(confidence))
  if len(predicted[file]['boxes']) == 0:
    no_predictions = True
  return predicted, no_predictions

In [ ]:
def plot_image_and_boxes_on_axis(image, boxes, ax, centers=True, trace=True, title="",color='blue'):
    ax.imshow(image[:, :, ::-1])
    for i, box_dict in enumerate(boxes):
        # box = box_dict['location']
        box = box_dict
        rect = patches.Rectangle((box[0], box[1]), box[2], box[3], fill=False, lw=2, color=color)

        ax.add_patch(rect)
        ax.set_axis_off()
    ax.title.set_text(title)
    return ax

In [ ]:
def calc_iou(gt_bbox, pred_bbox):
    '''
    This function takes the predicted bounding box and ground truth bounding box and
    return the IoU ratio
    '''
    x_topleft_gt, y_topleft_gt, x_bottomright_gt, y_bottomright_gt = gt_bbox
    x_topleft_p, y_topleft_p, x_bottomright_p, y_bottomright_p = pred_bbox

    if (x_topleft_gt > x_bottomright_gt) or (y_topleft_gt > y_bottomright_gt):
        raise AssertionError("Ground Truth Bounding Box is not correct")
    if (x_topleft_p > x_bottomright_p) or (y_topleft_p > y_bottomright_p):
        raise AssertionError("Predicted Bounding Box is not correct", x_topleft_p, x_bottomright_p, y_topleft_p,
                             y_bottomright_p)

    # if the GT bbox and predcited BBox do not overlap then iou=0
    if (x_bottomright_gt < x_topleft_p):
        # If bottom right of x-coordinate  GT  bbox is less than or above the top left of x coordinate of  the predicted BBox

        return 0.0
    if (
            y_bottomright_gt < y_topleft_p):  # If bottom right of y-coordinate  GT  bbox is less than or above the top left of y coordinate of  the predicted BBox

        return 0.0
    if (
            x_topleft_gt > x_bottomright_p):  # If bottom right of x-coordinate  GT  bbox is greater than or below the bottom right  of x coordinate of  the predcited BBox

        return 0.0
    if (
            y_topleft_gt > y_bottomright_p):  # If bottom right of y-coordinate  GT  bbox is greater than or below the bottom right  of y coordinate of  the predcited BBox

        return 0.0

    GT_bbox_area = (x_bottomright_gt - x_topleft_gt + 1) * (y_bottomright_gt - y_topleft_gt + 1)
    Pred_bbox_area = (x_bottomright_p - x_topleft_p + 1) * (y_bottomright_p - y_topleft_p + 1)

    x_top_left = np.max([x_topleft_gt, x_topleft_p])
    y_top_left = np.max([y_topleft_gt, y_topleft_p])
    x_bottom_right = np.min([x_bottomright_gt, x_bottomright_p])
    y_bottom_right = np.min([y_bottomright_gt, y_bottomright_p])

    intersection_area = (x_bottom_right - x_top_left + 1) * (y_bottom_right - y_top_left + 1)

    union_area = (GT_bbox_area + Pred_bbox_area - intersection_area)

    return intersection_area / union_area


In [ ]:
def get_model_scores(pred_boxes):
    """Creates a dictionary of from model_scores to image ids.
    Args:
        pred_boxes (dict): dict of dicts of 'boxes' and 'scores'
    Returns:
        dict: keys are model_scores and values are image ids (usually filenames)
    """
    model_score={}
    for img_id, val in pred_boxes.items():
        for score in val['scores']:
            if score not in model_score.keys():
                model_score[score]=[img_id]
            else:
                model_score[score].append(img_id)
    return model_score

In [ ]:
def calc_precision_recall(image_results):
    """Calculates precision and recall from the set of images
    Args:
        img_results (dict): dictionary formatted like:
            {
                'img_id1': {'true_pos': int, 'false_pos': int, 'false_neg': int},
                'img_id2': ...
                ...
            }
    Returns:
        tuple: of floats of (precision, recall)
    """
    true_positive=0
    false_positive=0
    false_negative=0
    precision = 0
    recall = 0
    for img_id, res in image_results.items():
        true_positive +=res['true_positive']
        false_positive += res['false_positive']
        false_negative += res['false_negative']
        try:
            precision = true_positive/(true_positive+ false_positive)
        except ZeroDivisionError:
            precision=0.0
        try:
            recall = true_positive/(true_positive + false_negative)
        except ZeroDivisionError:
            recall=0.0
    return (precision, recall)


In [ ]:
def get_single_image_results(gt_boxes, pred_boxes, iou_thr):
    """Calculates number of true_pos, false_pos, false_neg from single batch of boxes.
    Args:
        gt_boxes (list of list of floats): list of locations of ground truth
            objects as [xmin, ymin, xmax, ymax]
        pred_boxes (dict): dict of dicts of 'boxes' (formatted like `gt_boxes`)
            and 'scores'
        iou_thr (float): value of IoU to consider as threshold for a
            true prediction.
    Returns:
        dict: qty_ground_truth (int), qty_predicted (int), true positives (int), false positives (int), false negatives (int)
    """
    all_pred_indices = range(len(pred_boxes))
    all_gt_indices = range(len(gt_boxes))
    qty_predicted = len(pred_boxes)
    qty_ground_truth = len(gt_boxes)
    tp = 0
    fp = 0
    fn = 0
    output = "output"

    if len(all_pred_indices) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': fp, 'false_negative': fn}
    if len(all_gt_indices) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': fp, 'false_negative': fn}

    gt_idx_thr = []
    pred_idx_thr = []
    ious = []

    for ipb, pred_box in enumerate(pred_boxes):
        # print(ipb, pred_box)

        for igb, gt_box in enumerate(gt_boxes):
            # print(igb, gt_box)
            iou = calc_iou(gt_box, pred_box)

            if iou > iou_thr:
                gt_idx_thr.append(igb)
                pred_idx_thr.append(ipb)
                ious.append(iou)
    iou_sort = np.argsort(ious)[::1]
    if len(iou_sort) == 0:
        return {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': len(pred_boxes), 'false_negative': fn}
    else:
        gt_match_idx = []
        pred_match_idx = []
        for idx in iou_sort:
            gt_idx = gt_idx_thr[idx]
            pr_idx = pred_idx_thr[idx]
            # If the boxes are unmatched, add them to matches
            if (gt_idx not in gt_match_idx) and (pr_idx not in pred_match_idx):
                gt_match_idx.append(gt_idx)
                pred_match_idx.append(pr_idx)
        tp = len(gt_match_idx)
        fp = len(pred_boxes) - len(pred_match_idx)
        fn = len(gt_boxes) - len(gt_match_idx)
        output = {'qty_ground_truth' : qty_ground_truth, 'qty_predicted' : qty_predicted, 'true_positive': tp, 'false_positive': fp, 'false_negative': fn}
    return output

In [ ]:
def get_avg_precision_at_iou(image,gt_boxes, pred_bb, no_predictions = False, iou_thr=0.5):
  if (no_predictions):
    return {image:{'qty_ground_truth' : len(gt_boxes[image]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[image])}}

  model_scores = get_model_scores(pred_bb)
  # TODO: does not work with scores that are all equal
  #print("model_scores: ", model_scores)
  sorted_model_scores = sorted(model_scores.keys())
  # print("ggg sorted model scores: ", sorted_model_scores)
  # Sort the predicted boxes in descending order (lowest scoring boxes first):
  for img_id in pred_bb.keys():
        #print('img_id: ',img_id)
        arg_sort = np.argsort(pred_bb[img_id]['scores'])
        pred_bb[img_id]['scores'] = np.array(pred_bb[img_id]['scores'])[arg_sort].tolist()
        pred_bb[img_id]['boxes'] = np.array(pred_bb[img_id]['boxes'])[arg_sort].tolist()

  pred_boxes_pruned = deepcopy(pred_bb)

  precisions = []
  recalls = []
  model_thrs = []
  img_results = {}
  img_ids = []
  model_score_thr = 0
  # Loop over model score thresholds and calculate precision, recall
  for ithr, model_score_thr in enumerate(sorted_model_scores[:-1]):
    # On first iteration, define img_results for the first time:
    # print("Model score thr : ", model_score_thr)
    img_ids = gt_boxes.keys() if ithr == 0 else model_scores[model_score_thr]

  if len(img_ids) == 0:
    print("pruned img_ids from after score", image)
    return {image:{'qty_ground_truth' : len(gt_boxes[image]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[image])}}

  for img_id in img_ids:
    # return if the size of prediction is 0
    if len(pred_bb[img_id]['boxes']) == 0:
      return {img_id:{'qty_ground_truth' : len(gt_boxes[img_id]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[img_id])}}

    gt_boxes_img = gt_boxes[img_id]
    box_scores = pred_boxes_pruned[img_id]['scores']
    start_idx = 0

    for score in box_scores:
      if score <= 0:
        pred_boxes_pruned[img_id]
        start_idx += 1
      else:
        break
        # Remove boxes, scores of lower than threshold scores:
    pred_boxes_pruned[img_id]['scores'] = pred_boxes_pruned[img_id]['scores'][start_idx:]
    pred_boxes_pruned[img_id]['boxes'] = pred_boxes_pruned[img_id]['boxes'][start_idx:]
    # Recalculate image results for this image
    img_results[img_id] = get_single_image_results(gt_boxes_img, pred_boxes_pruned[img_id]['boxes'], iou_thr=0.25)
    # calculate precision and recall
    
    #if len(img_results) == 0 and len(pred_bb[img_id]['boxes']) == 0 and len(gt_boxes[img_id]) != 0:
    if len(img_results[img_id]) == 0 and len(gt_boxes[img_id]) != 0:
      print("get_avg_precision_at_iou all predictions were pruned")
      return {img_id:{'qty_ground_truth' : len(gt_boxes[img_id]), 'qty_predicted' : 0, 'true_positive': 0, 'false_positive': 0, 'false_negative': len(gt_boxes[img_id])}}

  prec, rec = calc_precision_recall(img_results)
  precisions.append(prec)
  recalls.append(rec)
  model_thrs.append(model_score_thr)
  precisions = np.array(precisions)
  recalls = np.array(recalls)
  prec_at_rec = []
  for recall_level in np.linspace(0.0, 1.0, 11):
    try:
      args = np.argwhere(recalls > recall_level).flatten()
      prec = max(precisions[args])
      #print(recalls, "Recall")
      #print(recall_level, "Recall Level")
      #print(args, "Args")
      #print(prec, "precision")
    except ValueError:
      prec = 0.0
    prec_at_rec.append(prec)
  avg_prec = np.mean(prec_at_rec)
  return img_results

In [ ]:
#  Felzenszwalb et al.
def non_max_suppression_slow(boxes, overlapThresh=0.3):
	# if there are no boxes, return an empty list
	if len(boxes) == 0:
		return []
	# initialize the list of picked indexes
	pick = []
	# grab the coordinates of the bounding boxes
	x1 = boxes[:,0]
	y1 = boxes[:,1]
	x2 = boxes[:,2]
	y2 = boxes[:,3]
	# compute the area of the bounding boxes and sort the bounding
	# boxes by the bottom-right y-coordinate of the bounding box
	area = (x2 - x1 + 1) * (y2 - y1 + 1)
	idxs = np.argsort(y2)
 	# keep looping while some indexes still remain in the indexes
	# list
	while len(idxs) > 0:

		# grab the last index in the indexes list, add the index
		# value to the list of picked indexes, then initialize
		# the suppression list (i.e. indexes that will be deleted)
		# using the last index
		last = len(idxs) - 1
		i = idxs[last]
		pick.append(i)
		suppress = [last]
    # loop over all indexes in the indexes list
		for pos in range(0, last):
			# grab the current index
			j = idxs[pos]
			# find the largest (x, y) coordinates for the start of
			# the bounding box and the smallest (x, y) coordinates
			# for the end of the bounding box
			xx1 = max(x1[i], x1[j])
			yy1 = max(y1[i], y1[j])
			xx2 = min(x2[i], x2[j])
			yy2 = min(y2[i], y2[j])
			# compute the width and height of the bounding box
			w = max(0, xx2 - xx1 + 1)
			h = max(0, yy2 - yy1 + 1)
			# compute the ratio of overlap between the computed
			# bounding box and the bounding box in the area list
			overlap = float(w * h) / area[j]
			# if there is sufficient overlap, suppress the
			# current bounding box
			if overlap > overlapThresh:
				suppress.append(pos)
		# delete all indexes from the index list that are in the
		# suppression list
		idxs = np.delete(idxs, suppress)
	# return only the bounding boxes that were picked
	return pick

In [ ]:
# load the predictions and ground_truth from already saved images, respectivelly from their xml files
def load_boxes_from_xmls(validated_xml_directory_path, display_results=False):
    print("load_boxes_from_xmls nr_directories: ",len(validated_xml_directory_path))
    results = []
    for hive_model,xml_files_path in validated_xml_directory_path.items():
        if display_results:
            print('load_boxes_from_xmls hive_model:',hive_model)
            print('load_boxes_from_xmls xml_files_path:',xml_files_path)

        xml_lst = []
        for filename in os.listdir(xml_files_path):
            if not filename.endswith('.xml'): continue
            fullname = os.path.join(xml_files_path, filename)
            xml_lst.append(fullname)

        # work on single image/xml
        for xml in xml_lst:
            etree = ET.parse(xml)
            root = etree.getroot()

            img_name = root.find('filename').text

            predicted = {}
            predicted[img_name] = {}
            predicted[img_name]['boxes'] = []
            predicted[img_name]['scores'] = []
            gt_bboxes = {}
            no_predictions = False

            p_bbxcoords = []
            gt_pbbxcoords = []
            scores = []

            objects = root.findall('object')
            score = 0.70

            for obj in objects:
                box = obj.find('bndbox')
                xmin = int(box.find('xmin').text)
                ymin = int(box.find('ymin').text)
                xmax = int(box.find('xmax').text)
                ymax = int(box.find('ymax').text)

                bbtype = obj.find('name').text

                if bbtype == 'detected_bee':
                    p_bbxcoords.append([xmin, ymin, xmax, ymax])
                    scores.append(score)
                    score += 0.0005
                elif bbtype == 'gt_bee':
                    gt_pbbxcoords.append([xmin, ymin, xmax, ymax])
                else:
                    print('Error object type at ', img_name)

            predicted[img_name]['boxes'] = np.array(p_bbxcoords)
            predicted[img_name]['scores'] = np.array(scores)
            gt_bboxes[img_name] = np.array(gt_pbbxcoords)

            if len(predicted[img_name]['boxes']) == 0:
                no_predictions = True

            #print('img_name: ',img_name, ', nr predicted: ', len(p_bbxcoords), 'nr gt: ', len(gt_pbbxcoords))

            res = get_avg_precision_at_iou(img_name, gt_bboxes, predicted, no_predictions)
            if display_results:
                print(res)
            results.append([hive_model,res])
    return results

In [ ]:
# calculate validation results per hive and save to file 
def calc_validation_results(output, trained_model, display_results=True):
  results_per_hive_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results_per_hive.csv'
  results_for_model_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results_for_model.csv'

  if os.path.exists(results_per_hive_file_path):
    os.remove(results_per_hive_file_path)
  if os.path.exists(results_for_model_file_path):
    os.remove(results_for_model_file_path)

  total_qty_ground_truth = output['qty_ground_truth'].sum()
  total_qty_predicted = output['qty_predicted'].sum()
  total_true_positive = output['true_positive'].sum()
  total_false_negative = output['false_negative'].sum()
  total_false_positive = output['false_positive'].sum()

  all_names = []
  #for name in output.hive:
  for name in output.hive_name:
    all_names.append(name)

  # Results per hive
  results_per_hive_file = open(results_per_hive_file_path, 'w')
  results_per_hive_file.write("trained_model,hive_name,qty_ground_truth,qty_predicted,true_positive,false_positive,false_negative,nr_imgs,precision,recall\n")

  for hive_name in list(set(all_names)):
    filtered_df = output.loc[output['hive_name'] == hive_name]
    true_positive = filtered_df['true_positive'].sum()
    false_positive = filtered_df['false_positive'].sum()
    false_negative = filtered_df['false_negative'].sum()
    nr_imgs = len(filtered_df)
    precision = true_positive/(true_positive+false_positive)
    recall = true_positive / (true_positive + false_negative)

    if display_results:
      print(hive_name)
      print("TP: {}".format(true_positive))
      print("FP: {}".format(false_positive))
      print("FN: {}".format(false_negative))
      print("{} images".format(nr_imgs))
      print("Precision: {}".format(precision))
      print("Recall: {}".format(recall))
      print("_____________________________________________________________")

    qty_ground_truth = filtered_df['qty_ground_truth'].sum()
    qty_predicted = filtered_df['qty_predicted'].sum()

    line_str = trained_model + ','
    line_str += hive_name + ','
    line_str += str(qty_ground_truth) + ','
    line_str += str(qty_predicted) + ','
    line_str += str(true_positive) + ','
    line_str += str(false_positive) + ','
    line_str += str(false_negative) + ','
    line_str += str(nr_imgs) + ','
    line_str += str(precision) + ','
    line_str += str(recall)
    results_per_hive_file.write("%s\n" % line_str)
  results_per_hive_file.close()

  # Aggregated resulsts for model
  total_precision = total_true_positive/(total_true_positive+total_false_positive)
  total_recall = total_true_positive / (total_true_positive + total_false_negative)

  if display_results:
    #print("Model name: {}".format(weights_path.split('/')[-1]))
    print("Model name: {}".format(trained_model))
    print("Overall TP: {}".format(total_true_positive))
    print("Overall FN: {}".format(total_false_negative))
    print("Overall FP: {}".format(total_false_positive))
    print("{} images".format(len(output)))
    print("Overall Precision: {}".format(total_precision))
    print("Overall Recall: {}".format(total_recall))
    print("_____________________________________________________________")
    print("_____________________________________________________________")

  results_for_model_file = open(results_for_model_file_path, 'w')
  results_for_model_file.write("trained_model,qty_ground_truth,qty_predicted,true_positive,false_positive,false_negative,nr_imgs,precision,recall\n")
  line_str = trained_model + ','
  line_str += str(total_qty_ground_truth) + ','
  line_str += str(total_qty_predicted) + ','
  line_str += str(total_true_positive) + ','
  line_str += str(total_false_positive) + ','
  line_str += str(total_false_negative) + ','
  line_str += str(len(output)) + ','
  line_str += str(total_precision) + ','
  line_str += str(total_recall)
  results_for_model_file.write("%s\n" % line_str)
  results_for_model_file.close()


In [ ]:
# if not-overwrite only load images from hives that have not already been validated, meaning, the xml files don't exist, the exisitng xmls/images are saved in a separated structure
# if overwrite, all files in the same structure
def get_imgs_paths(val_prefix,trained_model,hives, overwrite_xmls = True):
    image_list = []
    validation_xml_directory_path = {}
    validated_xml_directory_path = {}

    for hive in hives:
        write_folder = True
        dp = val_prefix + trained_model + '/' + hive
        if not os.path.exists(dp):
            validation_xml_directory_path[hive] = dp
            #print("created validation folder: ", validation_xml_directory_path[hive])
            os.makedirs(validation_xml_directory_path[hive])
        else:
            #print("folder already exists: ", dp)
            write_folder = overwrite_xmls

            if len(os.listdir(dp)) == 0:
                print("folder is empty", pd)
                write_folder = True

            # TODO: include images from partially worked folders, currently you need to delete the xml files from it to reprocess it completelly 

            if len(os.listdir(dp)) == 0 or write_folder:
                validation_xml_directory_path[hive] = dp
            else:
                validated_xml_directory_path[hive] = dp
    
        #img_path = '/content/gdrive/My Drive/beeTracking_img_labeling/' + hive + '/validate'
        img_path = '/content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/'+ hive + '/validate'
        for filename in os.listdir(img_path):
            if not filename.endswith('.jpg'): continue
            fullname = os.path.join(img_path, filename)
            if write_folder:
                #print("include folder ", dp)
                image_list.append(fullname)
    return image_list, validation_xml_directory_path, validated_xml_directory_path


In [ ]:
!ls /content/gdrive/My\ Drive/beeTracking_img_labeling/UnitedQueens/
!readlink -f /content/gdrive/My\ Drive/beeTracking_img_labeling/UnitedQueens/validate/

test  train  validate
/content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/UnitedQueens/validate


# Start validation

## Runtime

In [ ]:
print(cv2.__version__)

4.5.0-dev


In [ ]:
# hives = ['Chueried_Hempbox', 
#         'Chueried_Hive01', 'Chueried_Hive01_red70', 'Chueried_Hive01_green80', 
#         'ClemensRed', 'ClemensYellow',
#         'Doettingen_Hive1', 
#         'Echolinde', 'Echolinde_Night', 
#         'Erlen_Hive04_diagonalview', 'Erlen_Hive04_frontview', 'Erlen_Hive04_smartphone', 
#         'Erlen_Hive11', 'Erlen_Hive11_Night', 'Erlen_Hive11_grayscale', 'Erlen_Hive11_red70',
#         'Froh14', 'Froh23_TreeCavity', 'UnitedQueens']

hives = ['Doettingen_Hive1']

In [ ]:
len(hives)

1

In [ ]:
# trained_models = [#'individual_training/Chueried_Hive01',
#                   #'individual_training/Chueried_Hive01_green80',
#                   #'individual_training/Chueried_Hive01_red70',
#                   #'individual_training/Erlen_Hive11',
#                   'individual_training/Froh14',
#                   'individual_training/Froh23_TreeCavity',
#                   'on_qty_of_train_images/Echolinde_10',
#                   'on_qty_of_train_images/Echolinde_20',
#                   'on_qty_of_train_images/Echolinde_40',
#                   #'on_qty_of_train_images/Echolinde_80',
#                   'on_qty_of_train_images/Froh14_10',
#                   'on_qty_of_train_images/Froh14_20',
#                   'on_qty_of_train_images/Froh14_40',
#                   #'on_qty_of_train_images/Froh14_80',
#                   #'on_qty_of_train_images/Froh14_160',
#                   'single_hive/all_hives_trainP',
#                   'single_hive/Chueried_Hempbox',
#                   'single_hive/Chueried_Hive01',
#                   'single_hive/ClemensRed',
#                   'single_hive/ClemensYellow',
#                   'single_hive/Doettingen_Hive1',
#                   'single_hive/Echolinde',
#                   'single_hive/Echolinde_Night',
#                   'single_hive/Erlen_Hive04_diagonalview',
#                   'single_hive/Erlen_Hive04_frontview',
#                   'single_hive/Erlen_Hive04_smartphone',
#                   'single_hive/Erlen_Hive11',
#                   'single_hive/Erlen_Hive11_grayscale',
#                   'single_hive/Erlen_Hive11_Night',
#                   ##'single_hive/Erlen_Hive11_red70', #not trained yet
#                   'single_hive/Froh14',
#                   'single_hive/Froh23_TreeCavity',
#                   'single_hive/UnitedQueens',
#                   'staged_2/Chueried_Hive01_UnitedQueens',
#                   'staged_2/Erlen_Hive11_UnitedQueens',
#                   'staged_2/Froh14_20_Chueried_Hive01'
#                   ]

trained_models = ['single_hive/Doettingen_Hive1']

In [ ]:
# trained_models = ['single_hive/Froh14',
#                   'single_hive/Froh23_TreeCavity',
#                   'single_hive/UnitedQueens',
#                   'staged_2/Chueried_Hive01_UnitedQueens',
#                   'staged_2/Erlen_Hive11_UnitedQueens',
#                   'staged_2/Froh14_20_Chueried_Hive01'
#                   ]

In [ ]:
len(trained_models)

1

In [ ]:
#hive_model = 'Chueried_Hive01_red70'
CONFIG_PATH = '/content/gdrive/.shortcut-targets-by-id/17mTQanLeTbYrBmIInKmp3PVgqdMuRxV_/darknet/yolov4_bees.cfg'
labelsPath = '/content/gdrive/.shortcut-targets-by-id/17mTQanLeTbYrBmIInKmp3PVgqdMuRxV_/darknet/bee.names'  
#DB_PATH = '/content/gdrive/.shortcut-targets-by-id/1GEhPtvNA74kjnjpNH_3FtrySVXIIiXnm/yolo4_validation/bees.db'
DB_PATH = '/content/gdrive/My Drive/yolo4_validation_copy/bees.db'
PLOT = True #for local testing to inspect images
VALIDATION_XML_DIRECTORY_PATH_PREFIX = '/content/gdrive/My Drive/yolo4_validation_copy/'
overwrite_xmls = False
# arguments
args_confidence = 0.5
args_threshold = 0.3

In [ ]:
print(DB_PATH)
!readlink -f /content/gdrive/My\ Drive/yolo4_validation_copy/bees.db

/content/gdrive/My Drive/yolo4_validation_copy/bees.db
/content/gdrive/My Drive/yolo4_validation_copy/bees.db


In [ ]:
# load the class labels our YOLO model was trained on
LABELS = open(labelsPath).read().strip().split("\n")
# initialize a list of colors to represent each possible class label
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")
# derive the paths to the YOLO weights and model configuration

In [ ]:
conn = sqlite3.connect(DB_PATH)
c = conn.cursor()

In [ ]:
query_str = "select hive_name, count(distinct(filename)), count(*) from validation_ground_truth where hive_name in ("
for hive in hives:
  query_str += "\'" + hive + "\',"
query_str = query_str[0:len(query_str)-1]
query_str += ") group by hive_name"

print(query_str)

c.execute(str(query_str))
#print(c.fetchone())

test = c.fetchall()

for row in test:
  print(row)

select hive_name, count(distinct(filename)), count(*) from validation_ground_truth where hive_name in ('Doettingen_Hive1') group by hive_name
('Doettingen_Hive1', 40, 1301)


## Loop over models

In [ ]:
!ls /content/gdrive/My\ Drive/yolo_models/

 backup_yolo3   backup_yolo4  'BeeWatchAI Documents'   cfg_files


In [ ]:
# Create directory to save images
from pathlib import Path
video_name = "Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4"
save_path = "/content/gdrive/My Drive/cropped_bees/" + video_name + '/'
print(save_path)
Path(save_path).mkdir(parents=True, exist_ok=True)

/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/


In [ ]:
!ls /content/gdrive/My\ Drive/cropped_bees/
#!rm -rf /content/gdrive/My\ Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4

0_LABELED_Doettingen
0_LABELED_Echolinde
Chueried_Hempbox
Chueried_Hive01
Chueried_Hive01_small
Doettingen_Hive1
Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4
Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4-20201023T065307Z-001.zip
Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4-20201023T065307Z-002.zip
Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4_first_5015_images_updated_labels.zip
Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4_first_5015_images.zip
Doettingen_Hive_1_M_Rec_20200427_132404_540_M_second_split.mp4
Doettingen_Hive_1_M_Rec_20200427_132404_540_Mv2.mp4.zip
Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4
Doettingen_Hive_1_M_Rec_20200427_171545_540_M.mp4
Doettingen_Hive1_small
Doettingen_Hive1.zip
Echolinde
Echolinde_Hive_11_Rec_20200603_091212_410_M.mp4
Echolinde.zip
Erlen_Hive11
Erlen_Hive11.zip
wtf
ziG6xpxI


In [ ]:
if 5 % 1 == 1:
  print(True)

In [ ]:
firstModel = True
for tm in trained_models:
    print("Start model: ", tm)
    start_model_time = gmtime()
    # Model configuration
    trained_model = tm
    weights_path = '/content/gdrive/.shortcut-targets-by-id/1Vx5gmORtJ2Q7Eprs3Ei4aMwwfOsKp4M9/yolo_models/backup_yolo4/' + trained_model  + '/yolov4_bees_best.weights'
    results_file_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model + '/validation_results.csv'

    # load our YOLO object detector trained on your dataset 
    # and determine only the *output* layer names that we need from YOLO
    print("[INFO] loading YOLO from disk...")
    net = cv2.dnn.readNetFromDarknet(CONFIG_PATH, weights_path)
    ln = net.getLayerNames()
    ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    start_predictions_time = gmtime()

    # split in two functions:
    # 1. make the predictions of the current images
    # 2. load the predictions and ground_truth from already saved images, respectivelly from their xml files
    # gt_bboxes = dict { truncated_filename : list[[xmin, ymin, xmax, ymax]]}


    results = []
    PLOT = False

    # 1. make the predictions of the current images
    vidcap = cv2.VideoCapture("/content/gdrive/My Drive/" + video_name)
    success,image = vidcap.read()
    count = 0
    boxes = []
    import csv
    save_location_csv = save_path + 'boxes.csv'
    with open(save_location_csv, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["image", "bee", "xmin", "ymin", "xmax", "ymax"])
        while success:
          # every n'th frame
          if count % 5 == 0:

            filename = str(count)
            bboxes, no_predictions = CNN_get_boxes_for_frame(filename, image)
            indices = non_max_suppression_slow(np.array(bboxes[filename]['boxes']))
            bboxes[filename]['boxes'] = np.array(bboxes[filename]['boxes'])[indices].tolist()
            bboxes[filename]['scores'] = np.array(bboxes[filename]['scores'])[indices].tolist()

            for j in range(0, len(bboxes[filename]['boxes'])):
              if image is not None:
                xmin = min(max(0, int(bboxes[filename]['boxes'][j][0])), image.shape[0])
                ymin = min(max(0, int(bboxes[filename]['boxes'][j][1])), image.shape[1])
                xmax = min(max(0, int(bboxes[filename]['boxes'][j][2])), image.shape[0])
                ymax = min(max(0, int(bboxes[filename]['boxes'][j][3])), image.shape[1])
                save_location = save_path + 'img_' + str(count) + '_bee' + str(j) + '.jpg'
                print(save_location)
                print(count, j, xmin, ymin, xmax, ymax)
                if(xmin < xmax and ymin < ymax): 
                  writer.writerow([count, j, xmin, ymin, xmax, ymax])
                  print(cv2.imwrite(save_location, image[ymin:ymax, xmin:xmax, :]))
          else:
            pass
          success,image = vidcap.read()
          count = count + 1

      
    end_predictions_time = gmtime()


Streaming output truncated to the last 5000 lines.
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee2.jpg
6795 2 1860 1409 1920 1727
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee3.jpg
6795 3 799 1410 873 1660
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee4.jpg
6795 4 813 1159 957 1449
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee5.jpg
6795 5 1896 1026 1920 1356
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee6.jpg
6795 6 1451 1029 1655 1275
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee7.jpg
6795 7 449 1024 553 1226
True
/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_6795_bee8.jpg
6795 8 1040 1065 1278

In [ ]:
%cd /content/gdrive/My\ Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4

/content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132404_540_M.mp4


In [ ]:
!ls -U | head -4

ls: cannot open directory '.': Input/output error


In [ ]:
!ls /content/gdrive/My\ Drive/cropped_bees/
!readlink -f /content/gdrive/My\ Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/
#!rm /content/gdrive/My\ Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4.zip
!zip -r -0 /content/gdrive/My\ Drive/fuck.zip /content/gdrive/My\ Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4

Streaming output truncated to the last 5000 lines.
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_465_bee48.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_465_bee49.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_465_bee50.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_465_bee51.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_465_bee53.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_470_bee0.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive_1_M_Rec_20200427_132858_540_M.mp4/img_470_bee1.jpg (stored 0%)
  adding: content/gdrive/My Drive/cropped_bees/Doettingen_Hive

In [ ]:
!readlink -f /content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/UnitedQueens/validate
!df -h /content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/UnitedQueens/validate
!readlink -f /content/gdrive/My\ Drive/beeTracking_img_labeling/UnitedQueens/validate/

/content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/UnitedQueens/validate
Filesystem      Size  Used Avail Use% Mounted on
drive            69G   39G   30G  57% /content/gdrive
/content/gdrive/.shortcut-targets-by-id/1ur2tAvROwtB0i02gEGKra35nD_jiasX9/beeTracking_img_labeling/UnitedQueens/validate


# Recalculate results files for all hives
for results per model and results per hive


In [ ]:
training_types = ['single_hive', 'individual_training']
#'on_qty_of_train_images',
#,'staged_2'

for training_type in training_types:
  for hive in hives:
    trained_model = training_type +  '/' + hive
    validation_xml_directory_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model
    results_file_path = validation_xml_directory_path + '/validation_results.csv'
    import pdb; pdb.set_trace()
    if not os.path.exists(validation_xml_directory_path):
      #print("no validation folder for model : ", trained_model)
      continue
    else:
      results_from_file = []
      results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)
      output = pd.DataFrame()
      output = pd.DataFrame(results_from_file)
      calc_validation_results(output=output, trained_model=trained_model, display_results=False)


> <ipython-input-68-f89805f69ad9>(11)<module>()
-> if not os.path.exists(validation_xml_directory_path):
(Pdb) print(not os.path.exists(validation_xml_directory_path))
True
(Pdb) print(validation_xml_directory_path)
/content/gdrive/My Drive/yolo4_validation_copy/single_hive/Chueried_Hempbox
(Pdb) q


BdbQuit: ignored

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
training_types = ['on_qty_of_train_images','staged_2']

for training_type in training_types:
  val_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + training_type
  onlydirectories = [f for f in listdir(val_path) if isdir(join(val_path, f))]
  for f in onlydirectories:
    trained_model = training_type +  '/' + f
    validation_xml_directory_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model
    results_file_path = validation_xml_directory_path + '/validation_results.csv'
    results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)
    output = pd.DataFrame()
    output = pd.DataFrame(results_from_file)
    import pdb; pdb.set_trace()
    calc_validation_results(output=output, trained_model=trained_model, display_results=False)


> <ipython-input-67-4d400d4b1cee>(8)<module>()
-> onlydirectories = [f for f in listdir(val_path) if isdir(join(val_path, f))]
(Pdb) q


BdbQuit: ignored

In [ ]:
from os import listdir
from os.path import isfile, isdir, join
training_types = ['on_qty_of_train_images','staged_2']

for training_type in training_types:
  val_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + training_type
  onlydirectories = [f for f in listdir(val_path) if isdir(join(val_path, f))]
  for f in onlydirectories:
    trained_model = training_type +  '/' + f
    validation_xml_directory_path = VALIDATION_XML_DIRECTORY_PATH_PREFIX + trained_model
    results_file_path = validation_xml_directory_path + '/validation_results.csv'
    results_from_file = genfromtxt(results_file_path, delimiter=',',dtype="U255,U50,U255,<i8,<i8,<i8,<i8,<i8",encoding="utf8", names = True)
    output = pd.DataFrame()
    output = pd.DataFrame(results_from_file)
    import pdb; pdb.set_trace()
    calc_validation_results(output=output, trained_model=trained_model, display_results=False)